In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-26 19:39:14--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-26 19:39:14 (9.80 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [39]:
import minsearch
import json
with open('simplified_national_parks_data.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


In [2]:
pip install elasticsearch==7.17.0


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Now create the index
es.indices.create(index=index_name, body=mapping)


/var/folders/y_/2fr4l7w5225c3d_xq68300q00000gn/T/ipykernel_68088/4178800714.py:5: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, body=mapping)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'parks'}

https://github.com/DataTalksClub/llm-zoomcamp/blob/main/02-open-source/huggingface-mistral-7b.ipynb

In [41]:
from elasticsearch import Elasticsearch, helpers

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Define your index schema (mapping) with additional fields
index_name = "parks"
mapping = {
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "activities": {"type": "text"},
            "topics": {"type": "text"},
            "weatherInfo": {"type": "text"},
            "fullName": {"type": "text"},
            "parkCode": {"type": "keyword"},
            "states": {"type": "text"},
            "contacts": {"type": "text"},  # Simplified contacts as text
            "url": {"type": "text"},
            "directionsInfo": {"type": "text"},
            "directionsUrl": {"type": "text"},
            "addresses": {"type": "text"},
            "images": {
                "type": "nested",  # Since images are a list of objects
                "properties": {
                    "title": {"type": "text"},
                    "url": {"type": "text"}
                }
            }
        }
    }
}

# Check if the index exists and create it if it doesn't
if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting and recreating it.")
    es.indices.delete(index=index_name)

# Create the index with the mapping
es.indices.create(index=index_name, body=mapping)


# Prepare the documents (in the format Elasticsearch expects)
docs = [
    {
        '_index': index_name,
        '_source': {
            'id': doc['id'],
            'url': doc['url'],
            'fullName': doc['fullName'],
            'parkCode': doc['parkCode'],
            'description': doc['description'],
            'activities': ' '.join(doc['activities']),  # Join activities into a single string
            'topics': ' '.join(doc['topics']),  # Join topics into a single string
            'states': doc['states'],
            'contacts': doc['contacts'],  # Using the simplified contacts as text
            'directionsInfo': doc['directionsInfo'],
            'directionsUrl': doc['directionsUrl'],
            'addresses': doc['addresses'],  # Simplified addresses as text
            'images': doc['images'],  # Nested images structure
            'weatherInfo': doc.get('weatherInfo', '')
        }
    } for doc in docs_raw
]

# Bulk insert the documents into Elasticsearch
helpers.bulk(es, docs)

print("Data indexed successfully.")


Index 'parks' already exists. Deleting and recreating it.
Data indexed successfully.


/var/folders/y_/2fr4l7w5225c3d_xq68300q00000gn/T/ipykernel_68088/1336480644.py:40: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, body=mapping)


In [45]:
def search_elasticsearch(query, filters=None, boost_fields=None, num_results=10):
    """
    Search Elasticsearch index with optional filters and boosting.

    Args:
        query (str): The search query string.
        filters (dict): Keyword fields to filter by.
        boost_fields (dict): Fields to apply boosting.
        num_results (int): Number of results to return.

    Returns:
        list of dict: List of matched documents with scores.
    """
    search_body = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "addresses", "url"]
                    }
                },
                "filter": [
                    {"term": {key: value}} for key, value in (filters or {}).items()
                ]
            }
        },
        "size": num_results
    }

    # Add boosting if specified
    if boost_fields:
        must_queries = [
            {
                "multi_match": {
                    "query": query,
                    "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "addresses", "url"]
                }
            }
        ]
        
        for field, boost in boost_fields.items():
            must_queries.append({
                "match": {
                    field: {
                        "query": query,
                        "boost": boost
                    }
                }
            })

        search_body['query']['bool']['must'] = must_queries

    # Perform the search
    try:
        response = es.search(index=index_name, body=search_body)
        return [{"_source": hit['_source'], "_score": hit['_score']} for hit in response['hits']['hits']]
    except Exception as e:
        print(f"Error occurred while searching: {e}")
        return []

# Example search query
results = search_elasticsearch(query="smoky mountains", boost_fields={"description": 2.0}, num_results=5)
print(results)


[{'_source': {'id': 'D9819727-18DF-4A84-BDDE-D4F2696DE340', 'url': 'https://www.nps.gov/grsm/index.htm', 'fullName': 'Great Smoky Mountains National Park', 'parkCode': 'grsm', 'description': "Ridge upon ridge of forest straddles the border between North Carolina and Tennessee in Great Smoky Mountains National Park. World renowned for its diversity of plant and animal life, the beauty of its ancient mountains, and the quality of its remnants of Southern Appalachian mountain culture, this is America's most visited national park. Plan your visit today!", 'activities': 'Arts and Culture Live Music Auto and ATV Scenic Driving Astronomy Stargazing Biking Road Biking Camping Backcountry Camping Car or Front Country Camping Horse Camping (see also Horse/Stock Use) Group Camping RV Camping Fishing Freshwater Fishing Fly Fishing Food Picnicking Guided Tours Self-Guided Tours - Auto Hands-On Citizen Science Hiking Backcountry Hiking Front-Country Hiking Horse Trekking Horse Camping (see also camp

/var/folders/y_/2fr4l7w5225c3d_xq68300q00000gn/T/ipykernel_68088/3033801004.py:56: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(index=index_name, body=search_body)


In [13]:
count_response = es.count(index=index_name)
print("Total documents in index:", count_response['count'])


Total documents in index: 0


In [13]:
%pip install --upgrade --quiet huggingface_hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2024.9.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

 ········


In [23]:
from langchain_huggingface import HuggingFaceEndpoint

llm_h = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/manoharyelubandi/.cache/huggingface/token
Login successful


In [81]:
from langchain_huggingface import HuggingFaceEndpoint
from elasticsearch import Elasticsearch

# Initialize Hugging Face client
llm_h = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Search function to query Elasticsearch
def search_elasticsearch(query, filters=None, num_results=2):
    search_body = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "states", "addresses", "url"]
                    }
                },
                "filter": [
                    {"term": {key: value}} for key, value in (filters or {}).items()
                ]
            }
        },
        "size": num_results
    }

    response = es.search(index="parks", **search_body)  # Updated to use parameter unpacking
    return [hit['_source'] for hit in response['hits']['hits']]

# Function to build the prompt for Hugging Face
def build_prompt(query, search_results):
    prompt_template = """
You're a park information assistant. Answer the QUESTION based on the CONTEXT from the parks database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Full Name: {doc['fullName']}\nDescription: {doc['description']}\nActivities: {doc['activities']}\nTopics: {doc['topics']}\nWeather Info: {doc['weatherInfo']}\nAddress: {doc['addresses']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# Function to call Hugging Face API
def llm(prompt):
    response = llm_h.invoke(prompt)  # Assuming this returns the generated text directly
    return response  # Directly return the response

# Main RAG function
def rag(query):
    # Perform search to get relevant documents
    search_results = search_elasticsearch(query)
    # Build prompt with search results
    prompt = build_prompt(query, search_results)
    # Get answer from Hugging Face model
    answer = llm(prompt)
    return answer

# Example query
query = 'What activities can I do at Acadia National Park?'
answer = rag(query)
print(answer)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/manoharyelubandi/.cache/huggingface/token
Login successful


ANSWER: At Acadia National Park, you can participate in various activities including biking, boating, camping, climbing, fishing, hiking, horse trekking, horseback riding, ice skating, paddling, wildlife watching, and birdwatching, among others. Additionally, you can attend cultural demonstrations, visit museums, exhibits, and shops, and explore the park's rich history and natural wonders. Please note that weather conditions can significantly impact your experience, so it's essential to check the park's website or contact the visitor center for the most up-to-date information.


In [96]:
query = 'how many national parks are there in Kentucky state ? give me only full names of them. '
answer = rag(query)
print(answer)



Full Name: Big South Fork National River and Recreation Area
Description: With its stunning gorges, rich history, and unparalleled opportunities for outdoor recreation, Big South Fork National River and Recreation Area offers visitors an experience unlike any other. Spanning parts of Tennessee and Kentucky, this unique park features rugged natural beauty, abundant wildlife, and a diverse cultural heritage.
Activities: Arts and Culture Cultural Demonstrations Canoeing Camping Backcountry Camping Canoe or Kayak Camping Car or Front Country Camping Horse Camping (see also Horse/Stock Use) RV Camping Fishing Freshwater Fishing Food Picnicking Guided Tours Self-Guided Tours - Walking Hiking Backcountry Hiking Front-Country Hiking Horse Trekking Horse Camping (see also camping) Horseback Riding Paddling Canoeing Kayaking Stand Up Paddleboarding Whitewater Rafting Junior Ranger Program Birdwatching Park Film Shopping Bookstore and Park Store
Topics: Agriculture and Farming Historic Structur

In [86]:
query = 'In which state channel islands is located ?'
answer = rag(query)
print(answer)



Answer:
The Channel Islands National Park is located in the state of California.


In [74]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [75]:
len(model.encode("This is a simple sentence"))


768